Il giro al mondo in 80 giorni: 

Problema del percorso minimo in un grafo.

3 città piu vicine

Pesi: 
    2 ore
    4 ore
    8 ore
    
    2 ore se è un altra nazione
    2 ore se ha + di 200k abitanti. 
    
Calcolare il tempo minimo del viaggio. 

librerie per gestire grafi e reti, capire algoritmo e implemetarlo, altre librerie... 

In [25]:
import numpy as np
import pandas as pd
import math as m
from collections import defaultdict
import heapq
import itertools
import tqdm 

In [3]:
file = 'italiancities.xlsx'
cities = pd.read_excel(file, index_col = 'id')

In [4]:
cities.isna().any()  #per verificare nel dataset totale se ci sono blanks. 

city_ascii    False
lat           False
lng           False
iso3          False
population    False
dtype: bool

https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#values-considered-missing

In [5]:
cities.head(4)

,city_ascii,lat,lng,iso3,population
id,,,,,
1380382862,Rome,41.8931,12.4828,ITA,2872800
1380724377,Milan,45.4669,9.1900,ITA,1351562
1380646673,Naples,40.8333,14.2500,ITA,972212
1380244627,Turin,45.0667,7.7000,ITA,870952


In [6]:
cities.dtypes

city_ascii     object
lat           float64
lng           float64
iso3           object
population      int64
dtype: object

In [7]:
cities.index[1:4]

Int64Index([1380724377, 1380646673, 1380244627], dtype='int64', name='id')

In [8]:
cities.city_ascii[:2]

id
1380382862     Rome
1380724377    Milan
Name: city_ascii, dtype: object

In [9]:
def dist2punti2(k, o):
    Rt=6378.388
    lat_A=(m.pi*k[0])/180
    lat_B=(m.pi*o[0])/180
    lon_A=(m.pi*k[1])/180
    lon_B=(m.pi*o[1])/180
    dl=abs(lon_A-lon_B)
    p = m.acos((m.sin(lat_B)*m.sin(lat_A))+(m.cos(lat_B)*m.cos(lat_A)*m.cos(dl)))
    if(lon_A>0 and lon_B>0): 
        if(lon_B > lon_A): 
            p=p
        else:
            p=2*m.pi-p
    if(lon_A<0 and lon_B<0): 
        if(lon_B > lon_A): 
            p=p
        else:
            p=2*m.pi-p
    if(lon_A>0 and lon_B<0):
        if((lon_A+abs(lon_B))<m.pi): 
            p=2*m.pi-p
        else:
            p=p
    if(lon_A<0 and lon_B>0):
        if((lon_B+abs(lon_A))<m.pi): 
            p=p
        else:
            p=2*m.pi-p
    return(p*Rt)

In [10]:
lat = [c for c in cities.lat]
lng = [c for c in cities.lng]
cit = [c for c in cities.city_ascii] #nodes
id_cit = [c for c in cities.index] #per creare table con indici unici

In [11]:
dict_coordinates= {i:[j, k] for i, j, k in zip(cit, lat, lng)}

In [12]:
dict_coordinates['Rome'] 

[41.8931, 12.4828]

In [13]:
dict_coordinates.keys()

dict_keys(['Rome', 'Milan', 'Naples', 'Turin', 'Palermo', 'Genoa', 'Bologna', 'Florence', 'Bari', 'Catania', 'Cagliari', 'Venice', 'Verona', 'Messina', 'Padova', 'Trieste', 'Brescia', 'Taranto', 'Parma', 'Prato', 'Modena', 'Reggio di Calabria', 'Reggio Emilia', 'Perugia', 'Livorno', 'Ravenna', 'Foggia', 'Rimini', 'Salerno', 'Ferrara', 'Latina', 'Monza', 'Giugliano in Campania', 'Pescara', 'Bergamo', 'Siracusa', 'Forli', 'Trento', 'Sassari', 'Vicenza', 'Terni', 'Barletta', 'Bolzano', 'Novara', 'Piacenza', 'Ancona', 'Andria', 'Catanzaro', 'Como', 'Busto Arsizio', 'Castellammare di Stabia', "L'Aquila", 'Potenza', 'Campobasso', 'Aosta', 'Udine', 'Lecce', 'Arezzo', 'Pisa', 'Eraclea', 'Brindisi', 'Treviso', 'Varese', 'Marsala', 'Caserta', 'Asti', 'Ragusa', 'Benevento', 'Civitavecchia', 'Savona', 'Molfetta', 'Crotone', 'Agrigento', 'Siena', 'Biella', 'Olbia', 'Macerata', 'Gorizia', 'Vibo Valentia', 'Ventimiglia', 'Pozzallo', 'Narni', 'San Vito dei Normanni', 'Urbino', 'Priverno', 'Muggia', 'L

In [14]:
dict_coordinates.values()

dict_values([[41.8931, 12.4828], [45.4669, 9.19], [40.8333, 14.25], [45.0667, 7.7], [38.1157, 13.3613], [44.4072, 8.934], [44.4939, 11.3428], [43.7714, 11.2542], [41.1253, 16.8667], [37.5027, 15.0873], [39.2224, 9.104], [45.4397, 12.3319], [45.4386, 10.9928], [38.1833, 15.55], [45.4064, 11.8778], [45.6361, 13.8042], [45.5389, 10.2203], [40.4181, 17.2408], [44.8015, 10.328], [43.8808, 11.0966], [44.6458, 10.9257], [38.1144, 15.65], [44.7, 10.6333], [43.1121, 12.3888], [43.55, 10.3167], [44.4178, 12.1994], [41.4584, 15.5519], [44.0594, 12.5683], [40.6806, 14.7597], [44.8353, 11.6199], [41.4676, 12.9037], [45.5836, 9.2736], [40.9319, 14.1956], [42.4643, 14.2142], [45.695, 9.67], [37.0692, 15.2875], [44.2225, 12.0408], [46.0667, 11.1167], [40.7267, 8.5592], [45.55, 11.55], [42.5667, 12.65], [41.32, 16.27], [46.5, 11.35], [45.45, 8.6167], [45.05, 9.7], [43.6167, 13.5167], [41.2317, 16.3083], [38.91, 16.5875], [45.8103, 9.0861], [45.612, 8.8518], [40.7, 14.4833], [42.3504, 13.39], [40.6333, 

In [15]:
Z = [_ for _ in dict_coordinates.keys()]


for a, k in  dict_coordinates.items():
    for b, o in  dict_coordinates.items():
        if a == b:
            pass
        else:
            dist2punti2(k, o)
            print (a, '-', b, ':', dist2punti2(k, o))

#print(Z)

Rome - Milan : 39598.60837723564
Rome - Naples : 188.9931727547731
Rome - Turin : 39553.238532742624
Rome - Palermo : 427.12790117858816
Rome - Genoa : 39674.91505208412
Rome - Bologna : 39772.647666365876
Rome - Florence : 39844.6892731753
Rome - Bari : 375.2782216422699
Rome - Catania : 537.1900450849964
Rome - Cagliari : 39664.278277693214
Rome - Venice : 39681.58578652911
Rome - Verona : 39664.08088196202
Rome - Messina : 488.6922569361534
Rome - Padova : 39682.458720790106
Rome - Trieste : 429.99701710639386
Rome - Brescia : 39631.821204187705
Rome - Taranto : 431.2102991162457
Rome - Parma : 39708.851452726645
Rome - Prato : 39828.111156454004
Rome - Modena : 39745.194696510254
Rome - Reggio di Calabria : 499.8080932616998
Rome - Reggio Emilia : 39730.0697192557
Rome - Perugia : 39940.670872995935
Rome - Livorno : 39820.86530063057
Rome - Ravenna : 39794.59419204613
Rome - Foggia : 259.7281330255044
Rome - Rimini : 241.2613843241284
Rome - Salerno : 233.42996872847678
Rome - Ferr

Bologna - Taranto : 663.4025504221548
Bologna - Parma : 39989.231288817566
Bologna - Prato : 40005.567977218336
Bologna - Modena : 40039.443356961834
Bologna - Reggio di Calabria : 796.0189011971923
Bologna - Reggio Emilia : 40015.8522680322
Bologna - Perugia : 175.283633092823
Bologna - Livorno : 39943.22315010757
Bologna - Ravenna : 68.59191665339897
Bologna - Foggia : 481.24654585959865
Bologna - Rimini : 108.99769438646048
Bologna - Salerno : 508.45657845490564
Bologna - Ferrara : 43.88408322239487
Bologna - Latina : 360.06621051232185
Bologna - Monza : 39873.60024647898
Bologna - Giugliano in Campania : 460.02692122404585
Bologna - Pescara : 323.7667385532924
Bologna - Bergamo : 39889.08975098044
Bologna - Siracusa : 890.6336097463757
Bologna - Forli : 63.24074119082101
Bologna - Trento : 39900.6103393786
Bologna - Sassari : 39599.287189618226
Bologna - Vicenza : 118.69412784845464
Bologna - Terni : 239.07344983350038
Bologna - Barletta : 534.849286611172
Bologna - Bolzano : 223.3

Messina - Busto Arsizio : 39081.43412110043
Messina - Castellammare di Stabia : 39781.80491735991
Messina - L'Aquila : 39577.77732822942
Messina - Potenza : 273.58945518850777
Messina - Campobasso : 39692.68457421824
Messina - Aosta : 38995.30663154213
Messina - Udine : 39178.49145447217
Messina - Lecce : 330.4851147615709
Messina - Arezzo : 39412.589241354166
Messina - Pisa : 39323.97878642016
Messina - Eraclea : 39212.974405636094
Messina - Brindisi : 342.2470195673932
Messina - Treviso : 39199.25362746402
Messina - Varese : 39062.11459004726
Messina - Marsala : 39800.426824374845
Messina - Caserta : 39739.084898292545
Messina - Asti : 39111.299510957884
Messina - Ragusa : 39919.42258903084
Messina - Benevento : 39742.4861111525
Messina - Civitavecchia : 39536.353658589025
Messina - Savona : 39174.54240794156
Messina - Molfetta : 347.6589435992574
Messina - Crotone : 169.5745170041954
Messina - Agrigento : 39877.58777349483
Messina - Siena : 39403.498649215304
Messina - Biella : 3904

Modena - Ventimiglia : 39795.32842803216
Modena - Pozzallo : 940.987491681293
Modena - Narni : 269.50385393249223
Modena - San Vito dei Normanni : 710.4198185138857
Modena - Urbino : 170.7861062489339
Modena - Priverno : 398.6753048221022
Modena - Muggia : 247.12991495152355
Modena - Loreto : 253.08076026343772
Modena - Amelia : 262.1012887143101
Modena - Avigliano : 585.7862985411306
Modena - Strongoli : 785.0413712124765
Modena - San Dorligo della Valle : 254.6260122198727
Modena - Montecastrilli : 255.14770509130693
Reggio di Calabria - Rome : 39576.78567182889
Reggio di Calabria - Milan : 39098.852301871855
Reggio di Calabria - Naples : 39750.895818028075
Reggio di Calabria - Turin : 39059.232699586086
Reggio di Calabria - Palermo : 39876.13932318142
Reggio di Calabria - Genoa : 39179.14549950131
Reggio di Calabria - Bologna : 39280.574863893395
Reggio di Calabria - Florence : 39346.63379209722
Reggio di Calabria - Bari : 351.03682428479163
Reggio di Calabria - Catania : 39992.4125

Rimini - Vicenza : 39892.19334414547
Rimini - Terni : 166.30483916666932
Rimini - Barletta : 429.73509572477656
Rimini - Bolzano : 39788.63384968111
Rimini - Novara : 39728.003937724505
Rimini - Piacenza : 39823.764458210746
Rimini - Ancona : 90.7092593623186
Rimini - Andria : 439.08980219069605
Rimini - Catanzaro : 663.8475446714265
Rimini - Como : 39740.04421404597
Rimini - Busto Arsizio : 39736.12713923885
Rimini - Castellammare di Stabia : 405.7532847418099
Rimini - L'Aquila : 201.5951058422798
Rimini - Potenza : 464.8554337752477
Rimini - Campobasso : 326.0127196747421
Rimini - Aosta : 39622.29981973948
Rimini - Udine : 229.49429871974397
Rimini - Lecce : 619.1219782483903
Rimini - Arezzo : 39989.792373393466
Rimini - Pisa : 39898.50136329652
Rimini - Eraclea : 175.8904505794271
Rimini - Brindisi : 583.5448703451714
Rimini - Treviso : 39895.21682349223
Rimini - Varese : 39723.24121919576
Rimini - Marsala : 39380.289231765
Rimini - Caserta : 363.20916722111224
Rimini - Asti : 39717

...

Arezzo - Giugliano in Campania : 340.47062038464117
Arezzo - Pescara : 220.54079350140654
Arezzo - Bergamo : 39772.67656971036
Arezzo - Siracusa : 768.2275804251681
Arezzo - Forli : 85.73499793339738
Arezzo - Trento : 39780.470143128594
Arezzo - Sassari : 39667.11849962461
Arezzo - Vicenza : 39842.68916926758
Arezzo - Terni : 117.92513518955329
Arezzo - Barletta : 432.82874782404514
Arezzo - Bolzano : 39735.843743847116
Arezzo - Novara : 39736.01563664559
Arezzo - Piacenza : 39828.946231329945
Arezzo - Ancona : 133.60029481026086
Arezzo - Andria : 441.1245480274574
Arezzo - Catanzaro : 642.1264423835094
Arezzo - Como : 39734.34269857249
Arezzo - Busto Arsizio : 39737.74323579696
Arezzo - Castellammare di Stabia : 375.40790729069147
Arezzo - L'Aquila : 174.82582494185365
Arezzo - Potenza : 452.0310272991246
Arezzo - Campobasso : 311.0238361163454
Arezzo - Aosta : 39635.33794607004
Arezzo - Udine : 309.2210185776374
Arezzo - Lecce : 625.6397044237032
Arezzo - Pisa : 39954.32236659714
Are

Caserta - L'Aquila : 39913.59940274672
Caserta - Potenza : 132.5952078969562
Caserta - Campobasso : 61.48660064533559
Caserta - Aosta : 39307.37266066166
Caserta - Udine : 39512.963420798646
Caserta - Lecce : 333.29352015444397
Caserta - Arezzo : 39741.80941523137
Caserta - Pisa : 39638.95549231127
Caserta - Eraclea : 39549.96945647357
Caserta - Brindisi : 307.8825732344395
Caserta - Treviso : 39536.74006987012
Caserta - Varese : 39386.113761975226
Caserta - Marsala : 39678.692453095144
Caserta - Asti : 39420.40008834471
Caserta - Ragusa : 461.7889937517842
Caserta - Benevento : 38.13583894896549
Caserta - Civitavecchia : 39836.31672721252
Caserta - Savona : 39477.233646604174
Caserta - Molfetta : 190.62891861165792
Caserta - Crotone : 324.3699674240458
Caserta - Agrigento : 39653.44258320623
Caserta - Siena : 39724.318504479816
Caserta - Biella : 39363.11806019634
Caserta - Olbia : 39671.42016556668
Caserta - Macerata : 39817.5657048064
Caserta - Gorizia : 39531.56224475384
Caserta - 

Crotone - Florence : 39361.16752346914
Crotone - Bari : 39848.22316721702
Crotone - Catania : 39826.39473636563
Crotone - Cagliari : 39384.358900910454
Crotone - Venice : 39266.67922987907
Crotone - Verona : 39207.88971721921
Crotone - Messina : 39907.01924808639
Crotone - Padova : 39250.97635847179
Crotone - Trieste : 39297.87833862958
Crotone - Brescia : 39161.141761488514
Crotone - Taranto : 148.93498419959656
Crotone - Parma : 39227.61233649507
Crotone - Prato : 39343.60437654432
Crotone - Modena : 39272.4533990023
Crotone - Reggio di Calabria : 39909.07283801792
Crotone - Reggio Emilia : 39252.398821311326
Crotone - Perugia : 39477.710878933474
Crotone - Livorno : 39321.31930611867
Crotone - Ravenna : 39355.86646164959
Crotone - Foggia : 39780.42305969294
Crotone - Rimini : 39405.43934383951
Crotone - Salerno : 39807.5628241777
Crotone - Ferrara : 39291.15209831892
Crotone - Latina : 39630.71059388458
Crotone - Monza : 39107.52403308852
Crotone - Giugliano in Campania : 39753.0966

Ventimiglia - Muggia : 527.1669313364305
Ventimiglia - Loreto : 484.9699118185556
Ventimiglia - Amelia : 413.88862109746583
Ventimiglia - Avigliano : 749.2574437003485
Ventimiglia - Strongoli : 934.4794470329712
Ventimiglia - San Dorligo della Valle : 534.6897073021827
Ventimiglia - Montecastrilli : 415.2811834287609
Pozzallo - Rome : 39466.92257088515
Pozzallo - Milan : 38995.10494637402
Pozzallo - Naples : 39616.90973187074
Pozzallo - Turin : 38971.774276066804
Pozzallo - Palermo : 39874.03576502555
Pozzallo - Genoa : 39087.10294920438
Pozzallo - Bologna : 39163.24242165866
Pozzallo - Florence : 39235.64433556809
Pozzallo - Bari : 519.5587971293107
Pozzallo - Catania : 88.5993460730642
Pozzallo - Cagliari : 39501.5054041236
Pozzallo - Venice : 39084.476523962134
Pozzallo - Verona : 39054.971277068216
Pozzallo - Messina : 173.28113957943606
Pozzallo - Padova : 39079.30998944454
Pozzallo - Trieste : 39081.35111865058
Pozzallo - Brescia : 39022.531292343294
Pozzallo - Taranto : 460.3407

Priverno - Strongoli : 410.3882159837372
Priverno - San Dorligo della Valle : 465.82502670177
Priverno - Montecastrilli : 39932.71966723995
Muggia - Rome : 39651.22271538102
Muggia - Milan : 39719.43672083915
Muggia - Naples : 532.0911016509064
Muggia - Turin : 39598.24333350603
Muggia - Palermo : 39242.738034040645
Muggia - Genoa : 39673.766684620154
Muggia - Bologna : 39849.65399541847
Muggia - Florence : 39792.043869309346
Muggia - Bari : 557.6463643008224
Muggia - Catania : 908.0745889978339
Muggia - Cagliari : 39270.134963540084
Muggia - Venice : 39963.26657215886
Muggia - Verona : 39859.49254271692
Muggia - Messina : 838.714623571272
Muggia - Padova : 39927.65128155082
Muggia - Trieste : 4.96754862900795
Muggia - Brescia : 39800.155892442315
Muggia - Taranto : 642.3012968273504
Muggia - Parma : 39792.61467925043
Muggia - Prato : 39791.65509787272
Muggia - Modena : 39829.46385013906
Muggia - Reggio di Calabria : 847.7599760037209
Muggia - Reggio Emilia : 39810.98944097714
Muggia -

In [16]:
I = defaultdict(lambda: defaultdict(lambda: 0))
for a, k in  dict_coordinates.items():
    for b, o in  dict_coordinates.items():
        if a == b:
            pass  
        else:
            I[a][b] = dist2punti2(k, o)
I = pd.DataFrame(I).T
I.fillna(0, inplace=True)

In [17]:
I.head() #da leggere da riga a colonna  ??

,Milan,Naples,Turin,Palermo,Genoa,Bologna,Florence,Bari,Catania,Cagliari,...,Urbino,Priverno,Muggia,Loreto,Amelia,Avigliano,Strongoli,San Dorligo della Valle,Montecastrilli,Rome
Rome,39598.608377,188.993173,39553.238533,427.127901,39674.915052,39772.647666,39844.689273,375.278222,537.190045,39664.278278,...,204.345780,75.135956,425.371050,195.292670,40002.874425,299.623176,485.365441,429.620508,84.261048,0.000000
Milan,0.000000,659.230708,39951.641817,888.259789,39956.911525,201.161630,249.742148,787.101157,1013.188367,39381.395961,...,335.012285,549.754142,357.157044,417.181670,414.578823,747.272803,945.736493,364.289832,409.499276,477.985388
Naples,39417.363058,0.000000,39365.014258,39764.575645,39486.971277,39604.756550,39666.995726,222.297384,377.746396,39602.784294,...,39728.345999,39962.720908,39544.502663,39781.567004,39831.874437,124.130985,296.670685,39542.495198,39826.732364,39887.600592
Turin,124.951948,711.579507,0.000000,905.518520,122.114781,294.801105,317.223619,863.964965,1043.491037,660.819971,...,420.070104,598.228794,478.350432,504.484303,470.888956,811.827967,1005.914813,485.637390,468.701608,523.355232
Palermo,39188.333976,312.018120,39171.075245,0.000000,39284.553175,39346.826405,39422.587707,450.036214,166.434738,39686.633833,...,39449.166690,39703.237147,833.855731,593.117182,39576.111519,354.847747,346.491746,836.694090,39566.365636,39649.465864


In [18]:
I[:1][:]

,Milan,Naples,Turin,Palermo,Genoa,Bologna,Florence,Bari,Catania,Cagliari,...,Urbino,Priverno,Muggia,Loreto,Amelia,Avigliano,Strongoli,San Dorligo della Valle,Montecastrilli,Rome
Rome,39598.608377,188.993173,39553.238533,427.127901,39674.915052,39772.647666,39844.689273,375.278222,537.190045,39664.278278,...,204.34578,75.135956,425.37105,195.29267,40002.874425,299.623176,485.365441,429.620508,84.261048,0.0


In [19]:
three_closest = defaultdict(lambda: defaultdict(lambda: 0))
for a, k in  dict_coordinates.items():
    for b, o in  dict_coordinates.items():
        if a == b:
            pass
        else:
            dist2punti2(k, o)
            three_closest[a][b] = dist2punti2(k, o)

In [20]:
three_closest

defaultdict(<function __main__.<lambda>()>,
            {'Rome': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'Milan': 39598.60837723564,
                          'Naples': 188.9931727547731,
                          'Turin': 39553.238532742624,
                          'Palermo': 427.12790117858816,
                          'Genoa': 39674.91505208412,
                          'Bologna': 39772.647666365876,
                          'Florence': 39844.6892731753,
                          'Bari': 375.2782216422699,
                          'Catania': 537.1900450849964,
                          'Cagliari': 39664.278277693214,
                          'Venice': 39681.58578652911,
                          'Verona': 39664.08088196202,
                          'Messina': 488.6922569361534,
                          'Padova': 39682.458720790106,
                          'Trieste': 429.99701710639386,
                          'Brescia': 39

In [22]:
ordered_by_distance = {k:dict(sorted(v.items(), key=lambda item: item[1])) for k,v in sorted(three_closest.items())}

In [61]:
ordered_by_distance

{'Agrigento': {'Ragusa': 110.8320201974975,
  'Pozzallo': 130.10093448166404,
  'Catania': 135.2891809097264,
  'Siracusa': 154.10766910365186,
  'Messina': 199.00599159575998,
  'Reggio di Calabria': 203.32100210571548,
  'Vibo Valentia': 267.9214305280722,
  'Catanzaro': 318.1541657241219,
  'Crotone': 367.6635942113698,
  'Strongoli': 374.6499182192706,
  'Castellammare di Stabia': 385.3278629003814,
  'Salerno': 388.8122238665197,
  'Naples': 396.39789966070896,
  'Giugliano in Campania': 406.60888709996533,
  'Potenza': 416.8727161910442,
  'Caserta': 423.15118188435446,
  'Avigliano': 423.5217249395473,
  'Benevento': 437.15562129683957,
  'Taranto': 469.50263829581183,
  'Campobasso': 482.339140208407,
  'Foggia': 491.9459710638892,
  'Andria': 495.8359065456897,
  'Barletta': 502.9099186468148,
  'Molfetta': 505.2422176927013,
  'Bari': 510.61053569651574,
  'San Vito dei Normanni': 515.7942137873728,
  'Lecce': 522.5677041193222,
  'Brindisi': 529.1886519305509,
  'Pescara': 5

In [24]:
for q, w in ordered_by_distance.items():
     for t, y in (itertools.islice(ordered_by_distance.items(), 3)):
             print(q,t)

Agrigento Agrigento
Agrigento Amelia
Agrigento Ancona
Amelia Agrigento
Amelia Amelia
Amelia Ancona
Ancona Agrigento
Ancona Amelia
Ancona Ancona
Andria Agrigento
Andria Amelia
Andria Ancona
Aosta Agrigento
Aosta Amelia
Aosta Ancona
Arezzo Agrigento
Arezzo Amelia
Arezzo Ancona
Asti Agrigento
Asti Amelia
Asti Ancona
Avigliano Agrigento
Avigliano Amelia
Avigliano Ancona
Bari Agrigento
Bari Amelia
Bari Ancona
Barletta Agrigento
Barletta Amelia
Barletta Ancona
Benevento Agrigento
Benevento Amelia
Benevento Ancona
Bergamo Agrigento
Bergamo Amelia
Bergamo Ancona
Biella Agrigento
Biella Amelia
Biella Ancona
Bologna Agrigento
Bologna Amelia
Bologna Ancona
Bolzano Agrigento
Bolzano Amelia
Bolzano Ancona
Brescia Agrigento
Brescia Amelia
Brescia Ancona
Brindisi Agrigento
Brindisi Amelia
Brindisi Ancona
Busto Arsizio Agrigento
Busto Arsizio Amelia
Busto Arsizio Ancona
Cagliari Agrigento
Cagliari Amelia
Cagliari Ancona
Campobasso Agrigento
Campobasso Amelia
Campobasso Ancona
Caserta Agrigento
Caserta

In [43]:
for index, row in I.head(n=1).iterrows():
    print('From: ',index, 'To:\\n',row)

From:  Rome To:\n Milan                      39598.608377
Naples                       188.993173
Turin                      39553.238533
Palermo                      427.127901
Genoa                      39674.915052
                               ...     
Avigliano                    299.623176
Strongoli                    485.365441
San Dorligo della Valle      429.620508
Montecastrilli                84.261048
Rome                           0.000000
Name: Rome, Length: 92, dtype: float64


In [44]:
closest_cities = I.apply(lambda x: heapq.nsmallest(4, x), axis=1)

In [45]:
closest_cities = I.apply(lambda x: heapq.nsmallest(4, x), axis=1)

In [46]:
closest_cities_df = pd.DataFrame(closest_cities.values.tolist())

In [47]:
closest_cities_df 

,0,1,2,3
0,0.0,58.893193,69.477826,75.135956
1,0.0,14.535888,45.205262,61.246306
2,0.0,24.640120,26.910421,46.220522
3,0.0,44.016814,62.124231,83.551400
4,0.0,91.554098,166.434738,178.955447
...,...,...,...,...
87,0.0,9.161799,12.045768,19.178222
88,0.0,13.167429,74.499999,80.156592
89,0.0,20.992975,129.039172,153.244896
90,0.0,352.736757,456.471605,483.269723


In [56]:
Y = np.argmin(I, axis=1, shape(1, -1), out=None)

SyntaxError: positional argument follows keyword argument (<ipython-input-56-d87d763128c3>, line 1)

In [49]:
# Initialize dictionary 
test_dict = {'gfg' : 1, 'is' : 4, 'best' : 6, 'for' : 7, 'geeks' : 3 } 
  
# Initialize K 
K = 2
  
# printing original dictionary 
print("The original dictionary is : " + str(test_dict)) 
  
# Smallest K values in Dictionary 
# Using nsmallest 
res = nsmallest(K, test_dict, key = test_dict.get) 
  
# printing result 
print("The minimum K value pairs are " + str(res)) 

The original dictionary is : {'gfg': 1, 'is': 4, 'best': 6, 'for': 7, 'geeks': 3}


NameError: name 'nsmallest' is not defined

In [50]:
c = 3
for c, p in 
close = nsmallest(3, test_dict, key = test_dict.get)

SyntaxError: invalid syntax (<ipython-input-50-6dee610d0a73>, line 2)

In [32]:
cities = [_ for _ in cit]
M = pd.DataFrame(np.zeros((len(cit), len(cit))), index= cities, columns= cities)

In [215]:
dict_distances

{'Rome': [41.8931, 12.4828],
 'Milan': [45.4669, 9.19],
 'Naples': [40.8333, 14.25],
 'Turin': [45.0667, 7.7],
 'Palermo': [38.1157, 13.3613],
 'Genoa': [44.4072, 8.934],
 'Bologna': [44.4939, 11.3428],
 'Florence': [43.7714, 11.2542],
 'Bari': [41.1253, 16.8667],
 'Catania': [37.5027, 15.0873],
 'Cagliari': [39.2224, 9.104],
 'Venice': [45.4397, 12.3319],
 'Verona': [45.4386, 10.9928],
 'Messina': [38.1833, 15.55],
 'Padova': [45.4064, 11.8778],
 'Trieste': [45.6361, 13.8042],
 'Brescia': [45.5389, 10.2203],
 'Taranto': [40.4181, 17.2408],
 'Parma': [44.8015, 10.328],
 'Prato': [43.8808, 11.0966],
 'Modena': [44.6458, 10.9257],
 'Reggio di Calabria': [38.1144, 15.65],
 'Reggio Emilia': [44.7, 10.6333],
 'Perugia': [43.1121, 12.3888],
 'Livorno': [43.55, 10.3167],
 'Ravenna': [44.4178, 12.1994],
 'Foggia': [41.4584, 15.5519],
 'Rimini': [44.0594, 12.5683],
 'Salerno': [40.6806, 14.7597],
 'Ferrara': [44.8353, 11.6199],
 'Latina': [41.4676, 12.9037],
 'Monza': [45.5836, 9.2736],
 'Giugl

In [43]:
M.loc['Rome']['Rome']

0.0

In [44]:
M['Rome']['Rome']

0.0

In [45]:
M.iloc[0][0] = 2